In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2  
import pyterrier as pt
import os
import numpy as np
import gc
print("Imports durchgelaufen")



###No Problemo

Imports durchgelaufen


In [2]:
# ----- PyTerrier einmalig initialisieren --------------------------
if not pt.started():        
    pt.init()  
print("PyTerrier gestartet")
###No Problemo

/tmp/ipykernel_255489/3210276138.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


PyTerrier gestartet


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipykernel_255489/3210276138.py:3: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [3]:
DATABASE = "longeval-web"
USER = "dis18"
HOST = "db"
PORT = "5432"
PASSWORD = "dis182425"

engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")


#No Problemo

In [4]:
# --- SQL Queries ---
query_topic = """
SELECT DISTINCT
    "Topic"."queryid" AS quer_id,    
    "Topic"."text_fr" AS query,
    "sub_collection" AS Zeitpunkt
FROM "Topic"
WHERE "sub_collection" = '2022-06';
"""

query_qrel = """
SELECT DISTINCT
    "Qrel"."relevance" AS relevance,
    "queryid" as quer_id,
    "docid" as doc_id
from "Qrel"
WHERE "sub_collection" = '2022-06';

"""

query_document = """
SELECT 
    "Document"."url" AS url,
    "docid" as doc_id
from "Document"
WHERE "sub_collection" = '2022-06';
"""

query_vorkommen = """
WITH url_counts AS (
    SELECT "url", COUNT(*) AS "Anzahl_Vorkommen"
    FROM "Document"  -- Fehlendes Anführungszeichen hier korrigiert
    WHERE "sub_collection" = '2022-06'
    GROUP BY "url"
)
SELECT "url", "Anzahl_Vorkommen" FROM url_counts;
"""


df_topic = pd.read_sql(query_topic, con=engine)
df_qrel = pd.read_sql(query_qrel, con=engine)
df_document = pd.read_sql(query_document, con=engine)
df_vorkommen = pd.read_sql(query_vorkommen, con=engine)

print("SQL eingelesen")
#No Problemo

SQL eingelesen


In [5]:
# --- Hilfsfunktion zur Speicheroptimierung ---
def optimize_df_types(df: pd.DataFrame) -> pd.DataFrame:
    """Optimizes DataFrame types in-place for memory."""
    for col in df.columns:
        col_type = df[col].dtype
        try:
            if col_type == 'float64':
                df[col] = df[col].astype(np.float32)
            elif col_type == 'int64':
                df[col] = pd.to_numeric(df[col], downcast='integer')
            elif col_type == 'object' and df[col].nunique() / len(df) < 0.5:
                 df[col] = df[col].astype('category')
        except Exception as e:
             # Optional: minimal error print if needed
             # print(f"Skipping {col}: {e}")
             pass # Ignore errors to keep it short
    return df

list_of_dfs = [df_topic, df_qrel, df_document, df_vorkommen]
optimized_list = [optimize_df_types(df) for df in list_of_dfs]

print("Datensätze Optimiert")

#No Problemo

Datensätze Optimiert


In [6]:
# --- Mergen der Dataframes ---

merged_df = pd.merge(df_topic, df_qrel, on='quer_id',how='inner' )

df_document['doc_id'] = df_document['doc_id'].str[3:]

merged_df = pd.merge(merged_df, df_document, on='doc_id',how='inner' )

merged_df = merged_df.drop_duplicates()

merged_df = pd.merge(merged_df, df_vorkommen, on='url', how='inner')

merged_df = merged_df.drop_duplicates()

print('Merges durchgeführt')

#No Problemo

Merges durchgeführt


In [7]:
# --- BM25 Daten laden ---

bm25_file = "./RUNS/longeval-web-fr-2022-06-BM25.gz"
bm_df = pt.io.read_results(bm25_file)

print('RUN geladen')

#No Problemo

RUN geladen


In [8]:
# --- Hilfsfunktion zur Speicheroptimierung bm_df ---
def optimize_df_types(df: pd.DataFrame) -> pd.DataFrame:
    """Optimizes DataFrame types in-place for memory."""
    for col in df.columns:
        col_type = df[col].dtype
        try:
            if col_type == 'float64':
                df[col] = df[col].astype(np.float32)
            elif col_type == 'int64':
                df[col] = pd.to_numeric(df[col], downcast='integer')
            elif col_type == 'object' and df[col].nunique() / len(df) < 0.5:
                 df[col] = df[col].astype('category')
        except Exception as e:
             # Optional: minimal error print if needed
             # print(f"Skipping {col}: {e}")
             pass # Ignore errors to keep it short
    return df

list_of_dfs = [bm_df]
optimized_list = [optimize_df_types(df) for df in list_of_dfs]
print("Run optimiert")
#No Problemo

Run optimiert


In [9]:
#---Rank einschränken & Rank Drop
bm_df = bm_df[bm_df['rank'] < 1000].copy()
bm_df.drop(columns=['rank'], inplace=True)

print(bm_df)

            qid    docno      score       name
0             3    27619  21.626516  pyterrier
1             3   684186  21.172441  pyterrier
2             3   637997  20.980433  pyterrier
3             3   430968  20.889212  pyterrier
4             3   673500  20.685411  pyterrier
...         ...      ...        ...        ...
23458308  75311     8861  13.525025  pyterrier
23458309  75311   547317  13.523710  pyterrier
23458310  75311  1554793  13.521506  pyterrier
23458311  75311   606629  13.521164  pyterrier
23458312  75311   597436  13.520813  pyterrier

[23458313 rows x 4 columns]


In [10]:
# --- Final Merge ---
merged_df.rename(columns={'quer_id': 'qid'}, inplace=True)
merged_bm_df = pd.merge(bm_df, merged_df, on='qid', how='inner')
print(merged_bm_df)

            qid    docno      score       name                 query  \
0             3    27619  21.626516  pyterrier  1ere guerre mondiale   
1             3    27619  21.626516  pyterrier  1ere guerre mondiale   
2             3    27619  21.626516  pyterrier  1ere guerre mondiale   
3             3    27619  21.626516  pyterrier  1ere guerre mondiale   
4             3    27619  21.626516  pyterrier  1ere guerre mondiale   
...         ...      ...        ...        ...                   ...   
14143058  75311     8861  13.525025  pyterrier   emploi  territorial   
14143059  75311   547317  13.523710  pyterrier   emploi  territorial   
14143060  75311  1554793  13.521506  pyterrier   emploi  territorial   
14143061  75311   606629  13.521164  pyterrier   emploi  territorial   
14143062  75311   597436  13.520813  pyterrier   emploi  territorial   

         zeitpunkt relevance doc_id  \
0          2022-06         0  21554   
1          2022-06         0  12307   
2          2022-06

In [11]:
merged_bm_df = merged_bm_df.drop_duplicates()

In [12]:
#--- Time-Tag Merge---
Time_Tags = pd.read_csv('./Topics_With_TimeTags.csv')
Time_Tags.rename(columns={'sub_collection': 'zeitpunkt'}, inplace=True)

print(Time_Tags)
Final_df = pd.merge(merged_bm_df, Time_Tags, on=['query', 'zeitpunkt'], how='inner')
print(Final_df)

                                          id  queryid  text_en  \
0       f72ba204-9502-4e2b-b5a3-886df465b726        3      NaN   
1       238e44f2-87f9-49fb-93d2-70d983da668d        4      NaN   
2       6e75448f-4832-4a59-acbf-e14722c3a48f        7      NaN   
3       dd3e414c-78fd-4df9-a4fc-56d2ad8950b8        8      NaN   
4       e25ebbf6-5250-4c29-b231-1dc1ede1683e        9      NaN   
...                                      ...      ...      ...   
153886  4a862cb1-1e72-4aac-a9da-d327eb050ebb    75421      NaN   
153887  53700ef9-a867-45bb-809c-967973f9e68a    75422      NaN   
153888  2d667e52-bbfd-486d-8647-19a2d40f1251    75423      NaN   
153889  97e0a5eb-c4ca-4123-a73a-ab116afa58b4    75424      NaN   
153890  e771d467-7848-4692-810a-220efc4fc4ee    75425      NaN   

                                       query zeitpunkt split  tag  
0                       1ere guerre mondiale   2022-06  test  0.2  
1                   1ere guerre mondiale ce2   2022-06  test  0.9  
2  

In [13]:
#Test der Columns 
Final_df.drop(columns=['id'], inplace=True)
Final_df.drop(columns=['text_en'], inplace=True)
Final_df.drop(columns=['split'], inplace=True)


Final_df = Final_df.drop_duplicates()

Final_df['tag'].unique()

array([0.2, 0.9, 0.5, 0.1, 0.3, 0.8, 0.7, 0. , 1. ])

In [14]:
#Final_df['Kombinierter_Score']  = (1 * Final_df['score']) + (1 * Final_df['Anzahl_Vorkommen']) + (1 * Final_df['tag']) 
Final_df['Kombinierter_Score']  = Final_df['score']

Final_df_for_evaluation = Final_df[['qid', 'doc_id', 'Kombinierter_Score', 'Anzahl_Vorkommen', 'tag']].rename(columns={
    'doc_id': 'docno',             
    'Kombinierter_Score': 'score',
    'qid': 'query_id'
})

print(Final_df_for_evaluation)
print(df_qrel)

         query_id  docno      score  Anzahl_Vorkommen  tag
0               3  21554  21.626516                 1  0.2
1               3  12307  21.626516                 1  0.2
2               3  21188  21.626516                 1  0.2
3               3  17509  21.626516                 1  0.2
4               3  14290  21.626516                 2  0.2
...           ...    ...        ...               ...  ...
14143058    75311  12400  13.525025                 1  0.2
14143059    75311  12400  13.523710                 1  0.2
14143060    75311  12400  13.521506                 1  0.2
14143061    75311  12400  13.521164                 1  0.2
14143062    75311  12400  13.520813                 1  0.2

[14143063 rows x 5 columns]
      relevance quer_id doc_id
0             0   17314   7967
1             0   14495  28630
2             0    7480  18789
3             0    3000  19163
4             1    5972   1869
...         ...     ...    ...
48857         1   19771   4233
48858         0

In [15]:
#Hyperparametertuning
import numpy as np
import itertools
df_qrel['relevance'] = df_qrel['relevance'].astype(int)
df_qrel = df_qrel.rename(columns={
    "quer_id":      "query_id",   
})
param_grid = {
    'w_score': [1000, 3000, 10000, 100000000], 
    'w_anzahl': [1000, 3000, 10000, 100000000], 
    'w_tag': [1000, 3000, 10000, 100000000]     
}

# Evaluationsmetrik, die wir optimieren wollen
eval_metric = "nDCG" 

result_list = {}

# Erzeuge alle möglichen Kombinationen von Gewichten
keys, values = zip(*param_grid.items())
weight_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

for weights in weight_combinations:

    w_score = weights['w_score']
    w_anzahl = weights['w_anzahl']
    w_tag = weights['w_tag']
    
    df = Final_df_for_evaluation.copy()
    df['score'] = df['score'].astype('float64')
    df['Anzahl_Vorkommen'] = df['Anzahl_Vorkommen'].astype('float64')
    df['tag'] = df['tag'].astype('float64')
    kombinierter_score_values = (w_score * df['score']) + (w_anzahl * df['Anzahl_Vorkommen']) + (w_tag * df['tag'])
    
    df['score'] = kombinierter_score_values 
    df.sort_values(by='score',ascending=False, inplace=True)
    kombinierter_score_values
    evaluation_metrics = ["map", "P_10", "ndcg"]
    evaluation_results = pt.Evaluate(df, df_qrel, evaluation_metrics)
    immutable_weights_key = tuple(sorted(weights.items()))
    result_list[immutable_weights_key] = evaluation_results
    print(f"Weights: {weights}, Results: {evaluation_results}")

print(result_list)

Weights: {'w_score': 1000, 'w_anzahl': 1000, 'w_tag': 1000}, Results: {'map': 0.19416178586720195, 'P_10': 0.04634052978104562, 'ndcg': 0.20577835751041382}
Weights: {'w_score': 1000, 'w_anzahl': 1000, 'w_tag': 3000}, Results: {'map': 0.19416178586720192, 'P_10': 0.046340529781045636, 'ndcg': 0.20577835751041382}
Weights: {'w_score': 1000, 'w_anzahl': 1000, 'w_tag': 10000}, Results: {'map': 0.19416178586720195, 'P_10': 0.04634052978104566, 'ndcg': 0.20577835751041387}
Weights: {'w_score': 1000, 'w_anzahl': 1000, 'w_tag': 100000000}, Results: {'map': 0.19416178586720145, 'P_10': 0.046340529781045615, 'ndcg': 0.20577835751041337}
Weights: {'w_score': 1000, 'w_anzahl': 3000, 'w_tag': 1000}, Results: {'map': 0.19416178586720195, 'P_10': 0.04634052978104563, 'ndcg': 0.2057783575104137}
Weights: {'w_score': 1000, 'w_anzahl': 3000, 'w_tag': 3000}, Results: {'map': 0.19416178586720192, 'P_10': 0.046340529781045636, 'ndcg': 0.2057783575104138}
Weights: {'w_score': 1000, 'w_anzahl': 3000, 'w_tag

In [16]:
weight_combinations

[{'w_score': 1000, 'w_anzahl': 1000, 'w_tag': 1000},
 {'w_score': 1000, 'w_anzahl': 1000, 'w_tag': 3000},
 {'w_score': 1000, 'w_anzahl': 1000, 'w_tag': 10000},
 {'w_score': 1000, 'w_anzahl': 1000, 'w_tag': 100000000},
 {'w_score': 1000, 'w_anzahl': 3000, 'w_tag': 1000},
 {'w_score': 1000, 'w_anzahl': 3000, 'w_tag': 3000},
 {'w_score': 1000, 'w_anzahl': 3000, 'w_tag': 10000},
 {'w_score': 1000, 'w_anzahl': 3000, 'w_tag': 100000000},
 {'w_score': 1000, 'w_anzahl': 10000, 'w_tag': 1000},
 {'w_score': 1000, 'w_anzahl': 10000, 'w_tag': 3000},
 {'w_score': 1000, 'w_anzahl': 10000, 'w_tag': 10000},
 {'w_score': 1000, 'w_anzahl': 10000, 'w_tag': 100000000},
 {'w_score': 1000, 'w_anzahl': 100000000, 'w_tag': 1000},
 {'w_score': 1000, 'w_anzahl': 100000000, 'w_tag': 3000},
 {'w_score': 1000, 'w_anzahl': 100000000, 'w_tag': 10000},
 {'w_score': 1000, 'w_anzahl': 100000000, 'w_tag': 100000000},
 {'w_score': 3000, 'w_anzahl': 1000, 'w_tag': 1000},
 {'w_score': 3000, 'w_anzahl': 1000, 'w_tag': 3000}